In [1]:
import numpy as np
import torch
from torch import tensor
from torch import nn
import matplotlib.pyplot as plt

import deepquantum.photonic as dqp
from  deepquantum.photonic.fock.ops import Displacement, BeamSplitter



# 基本操作

## QC Example 1

In [2]:
dir(dqp)

['QumodeCircuit',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'circuit',
 'fock',
 'gaussian',
 'state']

In [3]:
state = dqp.state.FockState(batch_size=3, n_modes=2, cutoff=15, pure=False, dtype=torch.complex128) 

cir = dqp.QumodeCircuit(batch_size=3, n_modes=2, backend='fock')
cir.displace(r=tensor(1.0), phi=tensor(2.0), mode=0)
cir.displace(r=tensor(2.0), phi=tensor(3.0), mode=1)
cir.beam_split(theta=tensor(0.5), phi=tensor(0.0), mode1=0, mode2=1)
state = cir(state) 

samples = state.homodyne_measure(phi=0., mode=0, shots=1) 


print('measurements:', samples)
print('collapsed state:', state.tensor)
print('collapsed state:', state.tensor.shape)

measurements: tensor([0.3103, 1.2783, 1.0641], dtype=torch.float64)
collapsed state: tensor([[[[[ 1.4703e-02-1.7364e-18j, -2.8481e-02-1.0051e-02j,
             3.4153e-02+2.7536e-02j,  ...,
            -2.2807e-03-3.0561e-03j,  7.1943e-04+2.0162e-03j,
            -6.1983e-05-1.1918e-03j],
           [-2.8481e-02+1.0051e-02j,  6.2043e-02+9.8334e-18j,
            -8.4983e-02-2.9991e-02j,  ...,
             6.5072e-03+4.3609e-03j, -2.7720e-03-3.4138e-03j,
             9.3482e-04+2.2663e-03j],
           [ 3.4153e-02-2.7536e-02j, -8.4983e-02+2.9991e-02j,
             1.3090e-01+2.8568e-18j,  ...,
            -1.1021e-02-2.8278e-03j,  5.4471e-03+3.3362e-03j,
            -2.3760e-03-2.6524e-03j],
           ...,
           [-2.2807e-03+3.0561e-03j,  6.5072e-03-4.3609e-03j,
            -1.1021e-02+2.8278e-03j,  ...,
             9.8901e-04-2.9956e-20j, -5.3068e-04-1.6322e-04j,
             2.5734e-04+1.7199e-04j],
           [ 7.1943e-04-2.0162e-03j, -2.7720e-03+3.4138e-03j,
             5.44

## QC Example 2

In [10]:
state = dqp.state.FockState(batch_size=3, n_modes=2, cutoff=15, pure=True, dtype=torch.complex128) 

cir = dqp.QumodeCircuit(batch_size=3, n_modes=2, backend='fock')

cir.add(Displacement(mode=0))
cir.operators[0].set_params(r=tensor(1.0), phi=tensor(2.0))
bs = BeamSplitter(mode1=0, mode2=1)
cir.add(bs)
bs.set_params(theta=tensor(0.2), phi=tensor(0.0))
print('theta', 'phi', bs.theta, bs.phi)
bs.set_params(theta=tensor(0.5), phi=tensor(0.5))
print('theta', 'phi', bs.theta, bs.phi)
state = cir(state) 

samples = state.homodyne_measure(phi=0., mode=0, shots=1) 


print('measurements:', samples)



theta phi tensor(0.2000) tensor(0.)
theta phi tensor(0.5000) tensor(0.5000)
measurements: tensor([-1.1711, -1.4641, -0.3149], dtype=torch.float64)


## QML Example 1 

In [9]:
x = torch.tensor([[1.7, 2.1, -4.0], 
                  [-1.1, 1.3, 2.0]])

encoding_cir = dqp.QumodeCircuit(batch_size=2, n_modes=2, backend='fock')
encoding_cir.displace(mode=0) 
encoding_cir.displace(mode=1)


var_cir = dqp.QumodeCircuit(batch_size=2, n_modes=2, backend='fock')
var_cir.add(Displacement(mode=0))
var_cir.add(Displacement(mode=1))
var_cir.add(BeamSplitter(mode1=0, mode2=1))


# nn.Module.forwar(x) forward pass of the module

# 1. loop over encoding_cir to encoding classocal data into quantum state
for i, op in enumerate(encoding_cir.operators):
    op.set_params(r=x[:,i], phi=torch.tensor(0.0)) 


# state = encoding_cir() # start with a vacum state
# state = var_cir(state)

cir = encoding_cir + var_cir + encoding_cir + var_cir # data re-uploading
state = cir()

print('state tensor', state.tensor.shape, state.tensor.dtype)

yhat, _ = state.quad_expectation(phi=0., mode=0)
print(yhat.shape)

for pn, p in cir.named_parameters():
    print(pn, p.grad)
    
yhat.sum().backward()

for pn, p in cir.named_parameters():
    print(pn, p.grad)

state tensor torch.Size([2, 10, 10]) torch.complex64
torch.Size([2])
operators.2.r None
operators.2.phi None
operators.3.r None
operators.3.phi None
operators.4.theta None
operators.4.phi None
operators.2.r tensor(-0.1301)
operators.2.phi tensor(1.3199)
operators.3.r tensor(1.7545)
operators.3.phi tensor(0.6636)
operators.4.theta tensor(-4.4975)
operators.4.phi tensor(1.5183)


# CVQNN拟合单光子态

In [10]:
dir(dqp)

['QumodeCircuit',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'circuit',
 'fock',
 'gaussian',
 'state']

In [11]:
cutoff = 15


class CVQNN(nn.Module):
    """
    https://strawberryfields.ai/photonics/demos/run_quantum_neural_network.html
    """
    def __init__(self, batch_size):
        super().__init__()
        self.var_cir = dqp.QumodeCircuit(batch_size=batch_size, n_modes=1, backend='fock', cutoff=cutoff, dtype=torch.complex64)
        self._build_cir()
        
    def _build_cir(self):
        for i in range(5):
            self.var_cir.phase_shift(mode=0)
            self.var_cir.squeeze(mode=0)
            self.var_cir.phase_shift(mode=0)
            self.var_cir.displace(mode=0)
            self.var_cir.kerr(mode=0)

    def forward(self):
        # Applies layers of gates to the initial state
        state = self.var_cir()
        return  state.tensor.squeeze()


model = CVQNN(batch_size=1)
model.to(torch.device("cuda"))
print(model)


target_state = np.zeros(cutoff)
target_state[1] = 1
target_state = torch.tensor(target_state, dtype=torch.complex64, device="cuda")

yhat = model() # Note：这时候模型还没有参数, 一定要先跑一次forward，才能有参数
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  

history = []
for i in range(300):
    yhat = model()
    fidelity = torch.abs(torch.sum(torch.conj(yhat) * target_state)) ** 2
    modulus = torch.abs(torch.sum(torch.conj(yhat) * yhat)) ** 2
    loss = 1 - fidelity
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Step: {i} Loss: {loss:.4f} Fidelity: {fidelity:.4f} Modulus: {modulus:.4f}")
    history.append(loss.item())


plt.plot(history)
plt.show()
    


CVQNN(
  (var_cir): QumodeCircuit(
    (init_state): FockState(batch_size=1, n_modes=1, cutoff=15, dtype=torch.complex64)
    (operators): ModuleList(
      (0): PhaseShifter(mode=0)
      (1): Squeezing(mode=0)
      (2): PhaseShifter(mode=0)
      (3): Displacement(mode=0)
      (4): KerrInteraction(mode=0)
      (5): PhaseShifter(mode=0)
      (6): Squeezing(mode=0)
      (7): PhaseShifter(mode=0)
      (8): Displacement(mode=0)
      (9): KerrInteraction(mode=0)
      (10): PhaseShifter(mode=0)
      (11): Squeezing(mode=0)
      (12): PhaseShifter(mode=0)
      (13): Displacement(mode=0)
      (14): KerrInteraction(mode=0)
      (15): PhaseShifter(mode=0)
      (16): Squeezing(mode=0)
      (17): PhaseShifter(mode=0)
      (18): Displacement(mode=0)
      (19): KerrInteraction(mode=0)
      (20): PhaseShifter(mode=0)
      (21): Squeezing(mode=0)
      (22): PhaseShifter(mode=0)
      (23): Displacement(mode=0)
      (24): KerrInteraction(mode=0)
    )
  )
)
Step: 0 Loss: 0.9425 F

KeyboardInterrupt: 